# 利用字符级 RNN 破译密码

在此 notebook 中，我们将了解如何构建一个递归神经网络并训练它破译用特定的密码加密的字符串。

此练习将使你熟悉预处理技巧和模型构建技巧，当你开始构建更高级的机器翻译、文本摘要等模型时，这些技巧将很有用。

## 数据集
我们使用的数据集包含 10,000 个加密短语和每个加密短语的明文版本。

我们首先加载并熟悉该数据集。

In [1]:
import helper

codes = helper.load_data('cipher.txt')
plaintext = helper.load_data('plaintext.txt')

`codes` 和 `plaintext` 都是数组，每个元素是一个短语。前三个加密短语是：

In [2]:
codes[:5]

['YMJ QNRJ NX MJW QJFXY QNPJI KWZNY , GZY YMJ GFSFSF NX RD QJFXY QNPJI .',
 'MJ XFB F TQI DJQQTB YWZHP .',
 'NSINF NX WFNSD IZWNSL OZSJ , FSI NY NX XTRJYNRJX BFWR NS STAJRGJW .',
 'YMFY HFY BFX RD RTXY QTAJI FSNRFQ .',
 'MJ INXQNPJX LWFUJKWZNY , QNRJX , FSI QJRTSX .']

明文版本是：

In [3]:
plaintext[:5]

['THE LIME IS HER LEAST LIKED FRUIT , BUT THE BANANA IS MY LEAST LIKED .',
 'HE SAW A OLD YELLOW TRUCK .',
 'INDIA IS RAINY DURING JUNE , AND IT IS SOMETIMES WARM IN NOVEMBER .',
 'THAT CAT WAS MY MOST LOVED ANIMAL .',
 'HE DISLIKES GRAPEFRUIT , LIMES , AND LEMONS .']

## 模型概述：字符级 RNN
我们将使用的模型是字符级 RNN，因为密码似乎针对的是字符。对于机器翻译任务来说，单词级 RNN 更常用。

字符级 RNN 将指定特定字符的整数作为输入，并输出另一个整数。要使模型能正常运转，我们需要按照以下步骤预处理我们的数据集：
 1. 将每个字符分离成一个数组元素（并不是整个短语或单词作为数组元素）
 1. 标记化字符，将其从字母变成整数，反之亦然
 1. 填充字符串，使所有输入和输出都能符合矩阵格式

要可视化此流程，我们假设源序列（在此例中是 `codes`）或目标序列（在此例中是 `plaintext`）如下所示（一个字符串列表）：

<img src="list_1.png" />

因为此模型将在字符级操作，因此我们需要将每个字符串分离成字符列表（在此 notebook 中由标记器完成）：

<img src="list_2.png" />

然后，标记化流程会将每个字符变成整数。注意，使用单词级 RNN（在大多数机器翻译示例中）时，标记器会为每个单词（而不是每个字母）分配一个整数，每个单元表示的是一个单词，而不是字符。

<img src="list_3.png" />

大多数机器学习平台都要求输入是矩阵，而不是由列表组成的列表。要将输入变成矩阵，我们需要找到列表中最长的成员，并用 0 填充所有更短的序列。假设在此例子中，“and two”是最长的序列，矩阵将如下所示：

<img src="padded_list.png" />

## 预处理（实现）
要使神经网络能够对文本数据进行预测，首先需要将其转换为网络能理解的数据。“dog”等文本数据是一系列 ASCII 字符编码。因为神经网络是一系列乘法和加法运算，因此输入数据必须是数字。

我们可以将每个字符变成数字，或将每个单词变成数字，它们分别称为字符 ID 和单词 ID。字符 ID 用于为每个字符生成预测的字符级模型。单词级模型使用单词 ID 为每个单词生成文本预测。单词级模型一般学习效果更好。

使用 Keras 的 [`Tokenizer`](https://keras.io/preprocessing/text/#tokenizer) 函数将每个句子转换成单词 ID 序列。因为我们处理的是字符级数据，因此确保将 `char_level` 标记设为合适的值。然后对 x 应用标记器。

In [8]:
from keras.preprocessing.text import Tokenizer


def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    # TODO: Implement
    x_tk = Tokenizer(char_level=True)
    x_tk.fit_on_texts(x)

    return x_tk.texts_to_sequences(x), x_tk

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{' ': 1, 'e': 2, 'o': 3, 'i': 4, 's': 5, 'h': 6, 'r': 7, 'y': 8, 'u': 9, 'c': 10, 'n': 11, 't': 12, 'a': 13, 'p': 14, '.': 15, 'T': 16, 'q': 17, 'k': 18, 'w': 19, 'f': 20, 'x': 21, 'm': 22, 'v': 23, 'l': 24, 'z': 25, 'd': 26, 'g': 27, 'b': 28, 'j': 29, 'B': 30, 'J': 31, ',': 32}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [16, 6, 2, 1, 17, 9, 4, 10, 18, 1, 28, 7, 3, 19, 11, 1, 20, 3, 21, 1, 29, 9, 22, 14, 5, 1, 3, 23, 2, 7, 1, 12, 6, 2, 1, 24, 13, 25, 8, 1, 26, 3, 27, 1, 15]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [30, 8, 1, 31, 3, 23, 2, 1, 32, 1, 22, 8, 1, 17, 9, 4, 10, 18, 1, 5, 12, 9, 26, 8, 1, 3, 20, 1, 24, 2, 21, 4, 10, 3, 27, 7, 13, 14, 6, 8, 1, 19, 3, 11, 1, 13, 1, 14, 7, 4, 25, 2, 1, 15]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [16, 6, 4, 5, 1, 4, 5, 1, 13, 1, 5, 6, 3, 7, 12, 1, 5, 2, 11, 12, 2, 11, 10, 2, 1, 15]


### 填充（实现）
批处理单词 ID 序列时，每个序列都必须长度一样。因为句子的长度不一，我们可以向序列末尾添加填充内容，使其长度相同。

确保使用 Keras 的函数 [`pad_sequences`](https://keras.io/preprocessing/sequence/#pad_sequences) 在每个序列的**末尾**添加填充内容，使所有密码序列长度相同，所有明文序列长度也相同。

In [33]:
import numpy as np
from keras.preprocessing.sequence import pad_sequences


def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    # TODO: Implement
    # Find the length of the longest string in the dataset. 
    # Then, pass it to pad_sentences as the maxlen parameter
    max_length=max([len(txt) for txt in x])
    if length is None:
        length = max_length
    print("max length:", max_length)
    return pad_sequences(x, maxlen=length,padding='post')

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

max length: 54
Sequence 1 in x
  Input:  [16  6  2  1 17  9  4 10 18  1 28  7  3 19 11  1 20  3 21  1 29  9 22 14  5
  1  3 23  2  7  1 12  6  2  1 24 13 25  8  1 26  3 27  1 15]
  Output: [16  6  2  1 17  9  4 10 18  1 28  7  3 19 11  1 20  3 21  1 29  9 22 14  5
  1  3 23  2  7  1 12  6  2  1 24 13 25  8  1 26  3 27  1 15  0  0  0  0  0
  0  0  0  0]
Sequence 2 in x
  Input:  [30  8  1 31  3 23  2  1 32  1 22  8  1 17  9  4 10 18  1  5 12  9 26  8  1
  3 20  1 24  2 21  4 10  3 27  7 13 14  6  8  1 19  3 11  1 13  1 14  7  4
 25  2  1 15]
  Output: [30  8  1 31  3 23  2  1 32  1 22  8  1 17  9  4 10 18  1  5 12  9 26  8  1
  3 20  1 24  2 21  4 10  3 27  7 13 14  6  8  1 19  3 11  1 13  1 14  7  4
 25  2  1 15]
Sequence 3 in x
  Input:  [16  6  4  5  1  4  5  1 13  1  5  6  3  7 12  1  5  2 11 12  2 11 10  2  1
 15]
  Output: [16  6  4  5  1  4  5  1 13  1  5  6  3  7 12  1  5  2 11 12  2 11 10  2  1
 15  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  

### 预处理管道
此项目的重点是构建神经网络架构，因此我们不需要你创建预处理管道。我们已经提供了 `preprocess` 函数的实现代码。

In [34]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_code_sentences, preproc_plaintext_sentences, code_tokenizer, plaintext_tokenizer =\
    preprocess(codes, plaintext)

print('Data Preprocessed')

max length: 101
max length: 101
Data Preprocessed


In [35]:
preproc_code_sentences[0]

array([ 5, 14,  3,  1, 10,  2, 13,  3,  1,  2,  4,  1, 14,  3,  6,  1, 10,
        3,  8,  4,  5,  1, 10,  2, 25,  3, 11,  1, 20,  6,  9,  2,  5,  1,
       18,  1, 17,  9,  5,  1,  5, 14,  3,  1, 17,  8,  7,  8,  7,  8,  1,
        2,  4,  1, 13, 15,  1, 10,  3,  8,  4,  5,  1, 10,  2, 25,  3, 11,
        1, 19,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0], dtype=int32)

In [84]:
from keras.layers import GRU, Input, Dense, TimeDistributed
from keras.models import Model
from keras.layers import Activation
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

def simple_model_test(input_shape, output_sequence_length, code_vocab_size, plaintext_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param code_vocab_size: Number of unique code characters in the dataset
    :param plaintext_vocab_size: Number of unique plaintext characters in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Build the model
    learning_rate = 1e-3
    latent_dim = 256
    # Define an input sequence and process it.
    encoder_inputs = Input(shape=(input_shape[1:]), name='input')
    encoder = GRU(latent_dim, return_sequences=True, name='encoder')
    encoder_outputs = encoder(encoder_inputs)
    dense = Dense(plaintext_vocab_size, activation='softmax', name='dense')
    output = dense(encoder_outputs)
    model = Model(encoder_inputs,dense(encoder_outputs))
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model



def simple_model(input_shape, output_sequence_length, code_vocab_size, plaintext_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param code_vocab_size: Number of unique code characters in the dataset
    :param plaintext_vocab_size: Number of unique plaintext characters in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Build the model
    learning_rate = 1e-3
    input_seq = Input(input_shape[1:],name='input')
    rnn = GRU(64, return_sequences=True, name='encoder')(input_seq)
    logits = TimeDistributed(Dense(plaintext_vocab_size, name='dense'), name='timeDistributed')(rnn) #这里为什么要用TimeDistributed?

    model = Model(input_seq, Activation('softmax', name='activation')(logits))
    model.compile(loss=sparse_categorical_crossentropy,
                   optimizer=Adam(learning_rate),
                   metrics=['accuracy'])
    
    return model


# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_code_sentences, preproc_plaintext_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_plaintext_sentences.shape[-2], 1))
print(tmp_x.shape)
#print(tmp_x[0])

max length: 101
(10001, 101, 1)


In [85]:
# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    preproc_plaintext_sentences.shape[1],
    len(code_tokenizer.word_index)+1,
    len(plaintext_tokenizer.word_index)+1)
simple_rnn_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 101, 1)            0         
_________________________________________________________________
encoder (GRU)                (None, 101, 64)           12672     
_________________________________________________________________
timeDistributed (TimeDistrib (None, 101, 32)           2080      
_________________________________________________________________
activation (Activation)      (None, 101, 32)           0         
Total params: 14,752
Trainable params: 14,752
Non-trainable params: 0
_________________________________________________________________


In [86]:
# Train the neural network
simple_rnn_model = simple_model_test(
    tmp_x.shape,
    preproc_plaintext_sentences.shape[1],
    len(code_tokenizer.word_index)+1,
    len(plaintext_tokenizer.word_index)+1)
simple_rnn_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 101, 1)            0         
_________________________________________________________________
encoder (GRU)                (None, 101, 256)          198144    
_________________________________________________________________
output (Dense)               (None, 101, 32)           8224      
Total params: 206,368
Trainable params: 206,368
Non-trainable params: 0
_________________________________________________________________


In [87]:
simple_rnn_model.fit(tmp_x, preproc_plaintext_sentences, batch_size=32, epochs=4, validation_split=0.2)

Train on 8000 samples, validate on 2001 samples
Epoch 1/4
8000/8000 [==============================] - 130s 16ms/step - loss: 0.9813 - acc: 0.7292 - val_loss: 0.3560 - val_acc: 0.9181
Epoch 2/4
8000/8000 [==============================] - 129s 16ms/step - loss: 0.1955 - acc: 0.9587 - val_loss: 0.1120 - val_acc: 0.9752
Epoch 3/4
8000/8000 [==============================] - 129s 16ms/step - loss: 0.0874 - acc: 0.9806 - val_loss: 0.0668 - val_acc: 0.9841
Epoch 4/4
8000/8000 [==============================] - 129s 16ms/step - loss: 0.0555 - acc: 0.9876 - val_loss: 0.0470 - val_acc: 0.9891


In [88]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], plaintext_tokenizer))

`logits_to_text` function loaded.
T H E   L I M E   I S   H E R   L E A S T   L I K E D   F R U I T   ,   B U T   T H E   B A N A N A   I S   M Y   L E A S T   L I K E D   . <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


In [89]:
plaintext[0]

'THE LIME IS HER LEAST LIKED FRUIT , BUT THE BANANA IS MY LEAST LIKED .'

大功告成。该 RNN 能够学习这个基本字符级密码（是一个简单的[凯撒密码](https://en.wikipedia.org/wiki/Caesar_cipher)）。如果你想接受更大的密码学挑战，请参阅我们的[利用递归神经网络破译密码](https://greydanus.github.io/2017/01/07/enigma-rnn/)。